In [1]:
import os
import pandas as pd
import json
import traceback

In [2]:
from langchain_openai import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv() #reteriving data from .env file

True

In [4]:
KEY=os.getenv("MY_API_KEY") #getting key from local environmet 

In [72]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain_community.callbacks.manager import get_openai_callback
from langchain_core.output_parsers import JsonOutputParser #for output format
import PyPDF2

In [73]:
llm=ChatOpenAI( model_name="google/gemma-2-9b-it:free", base_url="https://openrouter.ai/api/v1",openai_api_key=KEY,temperature=0.4)

In [7]:
op=llm.invoke("hi")

In [8]:
print(op,"\n")
print(op.content)

content='\nHello! 👋  How can I help you today? 😊' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 10, 'total_tokens': 24, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'google/gemma-2-9b-it:free', 'system_fingerprint': None, 'id': 'gen-1754494948-QpflRA3RCToMXCedAIuM', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None} id='run--7e86d637-1a95-4ca7-a163-de8ae4016c26-0' usage_metadata={'input_tokens': 10, 'output_tokens': 14, 'total_tokens': 24, 'input_token_details': {}, 'output_token_details': {}} 


Hello! 👋  How can I help you today? 😊


In [9]:
RESPONSE_JSON={
    "1": {
        "no": "1",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    },
    
    "2": {
        "no": "2",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    },
    
    "3": {
        "no": "3",
        "mcq": "multiple choice questions",
        "options":{
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here"
        },
        
        "correct": "correct answer"
    }
    
    
}

In [10]:
number=5
subject="Machine Learning"
tone='simple'

In [49]:
#creating template
template="""
Text:{text}
You are a expert in making Multiple Choice Questions. Given the above text its your Job to create {number} multiple choice questions
for {subject} students with {tone} tone (difficulty level).
Make sure the questions are not repeated and all the questions to be conforming the text as well.
Make sure to format your questions in json format - RESPONSE_JSON as given below and use it as guide.
Ensure to only make {number} questions.
###RESPONSE_JSON
{RESPONSE_JSON}.
If you cannot make MCQ's with the given data of if there is no enough data given just ask them to provide more data.
Make sure to format your response as raw JSON only.Do not include Markdown formatting like ```json or any explanation text.
Just return the JSON object directly.
"""

In [50]:
quiz_generator_prompt=PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=template
)

In [51]:
output_parser = JsonOutputParser()

In [52]:
#now will start creating chains
quiz_chain=LLMChain(llm=llm,prompt=quiz_generator_prompt,
                    output_key='quiz',
                    verbose=True,
                    ) #as this template requires a json output we use output_parser

In [53]:
#next prompt to ensure correctness and difficulty level
template2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of teh question and give a complete analysis of the quiz if the students
will be able to unserstand the questions and answer them. Only use at max 50 words for complexity analysis. 
if the quiz is not at par with the cognitive and analytical abilities of the students,\
update tech quiz questions which needs to be changed  and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [54]:
quiz_evalution_prompt=PromptTemplate(
    input_variables=['subject','quiz'],
    template=template2
)

In [55]:
review_chain=LLMChain(llm=llm,prompt=quiz_evalution_prompt,output_key="review",verbose=True)

In [56]:
quiz_chain

LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['RESPONSE_JSON', 'number', 'subject', 'text', 'tone'], input_types={}, partial_variables={}, template="\nText:{text}\nYou are a expert in making Multiple Choice Questions. Given the above text its your Job to create {number} multiple choice questions\nfor {subject} students with {tone} tone (difficulty level).\nMake sure the questions are not repeated and all the questions to be conforming the text as well.\nMake sure to format your questions in json format - RESPONSE_JSON as given below and use it as guide.\nEnsure to only make {number} questions.\n###RESPONSE_JSON\n{RESPONSE_JSON}.\nIf you cannot make MCQ's with the given data of if there is no enough data given just ask them to provide more data.\nMake sure to format your response as raw JSON only.Do not include Markdown formatting like ```json or any explanation text.\nJust return the JSON object directly.\n"), llm=ChatOpenAI(client=<openai.resources.chat.completions.com

In [57]:
#using SequentialChain to put all together
generate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=['RESPONSE_JSON', 'number', 'subject', 'text', 'tone'],
                               output_variables=["quiz", "review"], verbose=True)


In [58]:
file_path=r'/Users/ganeshreddybodireddy/Desktop/genai_mcq_project/data.txt'

In [59]:
with open(file_path,'r') as file:
    TEXT=file.read()

In [60]:
print(TEXT)

chine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[4][5]

From a theoretical viewpoint, probably approximately cor

In [61]:
json.dumps(RESPONSE_JSON)

'{"1": {"no": "1", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"no": "2", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"no": "3", "mcq": "multiple choice questions", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [62]:
#setting up token usage
with get_openai_callback() as cb:
    response=generate_chain(
        {
        'text':TEXT,
        'number':number,
        'subject':subject,
        'tone':tone,
        'RESPONSE_JSON':json.dumps(RESPONSE_JSON)
        }
    )
    



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:chine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalise to unseen data, and thus perform tasks without explicit instructions.[1] Within a subdiscipline in machine learning, advances in the field of deep learning have allowed neural networks, a class of statistical algorithms, to surpass many previous machine learning approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine. The application of ML to business problems is known as predictive analytics.

Statistics and mathematical optimisation (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory da

In [63]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2731
Prompt Tokens:2085
Completion Tokens:646
Total Cost:0.0


In [64]:
quiz_str=response.get('quiz')

In [67]:
print((quiz_str))  # This shows hidden characters like \n or spaces



{"1": {"no": "1", "mcq": "What was the primary focus of early machine learning research?", "options": {"a": "Achieving artificial general intelligence", "b": "Developing expert systems", "c": "Solving practical problems with data", "d": "Understanding human thought processes"}, "correct": "c"}, "2": {"no": "2", "mcq": "Which of these is NOT a key characteristic of machine learning?", "options": {"a": "Learning from experience", "b": "Improving performance over time", "c": "Explicitly programmed rules", "d": "Making predictions"}, "correct": "c"}, "3": {"no": "3", "mcq": "Who is credited with coining the term 'machine learning'?", "options": {"a": "Alan Turing", "b": "John McCarthy", "c": "Arthur Samuel", "d": "Warren McCulloch"}, "correct": "c"}, "4": {"no": "4", "mcq": "What was a major challenge faced by early probabilistic systems in AI?", "options": {"a": "Lack of computational power", "b": "Difficulty in acquiring and representing data", "c": "Limited understanding of neural netw

In [68]:
x=json.loads(quiz_str)

In [70]:
x.values()

dict_values([{'no': '1', 'mcq': 'What was the primary focus of early machine learning research?', 'options': {'a': 'Achieving artificial general intelligence', 'b': 'Developing expert systems', 'c': 'Solving practical problems with data', 'd': 'Understanding human thought processes'}, 'correct': 'c'}, {'no': '2', 'mcq': 'Which of these is NOT a key characteristic of machine learning?', 'options': {'a': 'Learning from experience', 'b': 'Improving performance over time', 'c': 'Explicitly programmed rules', 'd': 'Making predictions'}, 'correct': 'c'}, {'no': '3', 'mcq': "Who is credited with coining the term 'machine learning'?", 'options': {'a': 'Alan Turing', 'b': 'John McCarthy', 'c': 'Arthur Samuel', 'd': 'Warren McCulloch'}, 'correct': 'c'}, {'no': '4', 'mcq': 'What was a major challenge faced by early probabilistic systems in AI?', 'options': {'a': 'Lack of computational power', 'b': 'Difficulty in acquiring and representing data', 'c': 'Limited understanding of neural networks', 'd

In [30]:
#lets create a dataframe
for keys,value in quiz_str.items():
    print(value['mcq']," \n" ,value['options'])
    print(value['correct'])

AttributeError: 'list' object has no attribute 'items'

In [248]:
quiz_table=[]
for keys,value in quiz_str.items():
    mcq=value['mcq']
    options=" | ".join(
        [
         f"{option} : {option_value}"
        for option,option_value in value['options'].items()
        ]
    )
    correct=value['correct']
    quiz_table.append({"MCQ":mcq,"Choices":options,"Correct":correct})

In [252]:
df=pd.DataFrame(quiz_table)

In [255]:
df

,MCQ,Choices,Correct
0,What is the primary goal of machine learning?,a : To create machines that can think like hum...,b
1,Which of the following is NOT a field where ma...,a : Natural Language Processing | b : Computer...,c
2,Who is credited with coining the term 'machine...,a : Alan Turing | b : Donald Hebb | c : Arthur...,c
3,What type of learning is used by a program tha...,a : Supervised learning | b : Unsupervised lea...,c
4,What is the main difference between machine le...,a : Machine learning programs are faster than ...,b


In [261]:
print(df.iloc[0])

MCQ            What is the primary goal of machine learning?
Choices    a : To create machines that can think like hum...
Correct                                                    b
Name: 0, dtype: object


In [267]:
for index, row in df.iterrows():
    print(row['MCQ'],"\n",row['Choices'],"\n", row["Correct"])

What is the primary goal of machine learning? 
 a : To create machines that can think like humans | b : To develop algorithms that learn from data and make predictions | c : To build robots that can perform physical tasks | d : To write computer programs that can solve logical puzzles 
 b
Which of the following is NOT a field where machine learning is applied? 
 a : Natural Language Processing | b : Computer Vision | c : Astrophysics | d : Medicine 
 c
Who is credited with coining the term 'machine learning'? 
 a : Alan Turing | b : Donald Hebb | c : Arthur Samuel | d : Tom M. Mitchell 
 c
What type of learning is used by a program that learns to play checkers by playing against itself? 
 a : Supervised learning | b : Unsupervised learning | c : Reinforcement learning | d : Transduction learning 
 c
What is the main difference between machine learning and traditional programming? 
 a : Machine learning programs are faster than traditional programs | b : Machine learning programs can le

In [271]:
#can convert to csv
df.to_csv("ml_mcqs.csv")